In [2]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce

In [3]:
plex_data = pd.read_csv('C:/Users/lzoeckler/Desktop/4plex/testing_data/other_dilutions_testing_input.csv',
                        skiprows=8, names=['patient_id', 'type', 'well', 'error',
                                           'HRP2_pg_ml', 'LDH_Pan_pg_ml',
                                           'LDH_Pv_pg_ml', 'CRP_ng_ml'])
plex_data = plex_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
plex_data['patient_id'] = plex_data['patient_id'].fillna(method='ffill')
plex_data.head()

,patient_id,type,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,CRP_ng_ml
0,calibrator (neat),reduced concentration (replicate 1),g12,NaN,590.00,9648.76,480.71,9535.24
1,calibrator (neat),reduced concentration (replicate 2),h12,NaN,masked,11332.92,516.29,8460.31
2,calibrator (1:3),reduced concentration (replicate 1),g11,"mo2, mo3",187.97,masked,masked,5871.99
3,calibrator (1:3),reduced concentration (replicate 2),h11,NaN,206.17,3547.27,166.06,2798.18
4,calibrator (1:9),reduced concentration (replicate 1),g10,NaN,61.94,1088.41,50.93,1146.64


In [4]:
samples_data = plex_data.loc[plex_data['patient_id'].str.contains('pa-')]
samples_data = samples_data.drop('type', axis=1)
samples_data['concentration'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[-1])
samples_data['patient_id'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[0])
samples_data = samples_data.loc[(samples_data['concentration'].str.contains('2500x|125000x|6250000x|312500000x'))]
samples_data = samples_data.sort_values(['patient_id', 'well'])
samples_data.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,CRP_ng_ml,concentration
75,pa-005,c1,NaN,< 334375000.00,16966230308.94,2946977316.25,< 8356250000.00,312500000x (1:312500000)
41,pa-005,c2,NaN,7872641.80,8227621.88,1487028.16,3534114.59,125000x (1:125000)
58,pa-005,d1,NaN,19073657.78,342607174.16,77321679.93,< 167125000.00,6250000x (1:6250000)
24,pa-005,d2,NaN,517929.76,1617228.81,28553.25,< 66850.00,2500x (1:2500)
76,pa-017,a1,NaN,< 334375000.00,10531449090.96,2651427271.66,< 8356250000.00,312500000x (1:312500000)


In [5]:
# threshhold values for various analytes
threshholds = {'HRP2_pg_ml': 330, 'LDH_Pan_pg_ml': 10514,
               'LDH_Pv_pg_ml': 497, 'CRP_ng_ml': 9574}

In [6]:
# constant to apply to the threshhold for different dilutions
dil_constants = {'2500x': 50, '125000x': 2500,
                 '6250000x': 125000, '312500000x': 6250000}

In [7]:
# positivity threshholds for various analytes
pos_threshholds = {'HRP2_pg_ml': 2.3, 'LDH_Pan_pg_ml': 47.8,
                   'LDH_Pv_pg_ml': 75.1, 'CRP_ng_ml': np.nan}

In [8]:
# Columns = [neat_above, neat_below, neat_LLQ, neat_ULQ, NA]
# Rows = [dil_above, dil_below, dil_LLQ, dil_ULQ, NA]
HRP2_matrix = np.array([['1:50', '1:50', '1:50', '1:50', '1:50'],
                        ['1:50', 'neat', 'neat', '1:50', 'fail'],
                        ['1:50', 'neat', 'neat', 'fail', 'fail'],
                        ['1:50', '1:50', 'fail', '1:50', '1:50'],
                        ['fail', '1:50', '1:50', 'fail', 'fail']])

LDH_Pan_matrix = np.array([['1:50', 'neat', 'neat', '1:50', '1:50'],
                           ['1:50', 'neat', 'neat', '1:50', 'fail'],
                           ['1:50', 'neat', 'neat', 'fail', 'fail'],
                           ['1:50', 'neat', 'fail', '1:50', '1:50'],
                           ['fail', 'neat', 'neat', 'fail', 'fail']])

LDH_Pv_matrix = np.array([['1:50', 'neat', 'neat', '1:50', '1:50'],
                          ['1:50', 'neat', 'neat', '1:50', 'fail'],
                          ['1:50', 'neat', 'neat', 'fail', 'fail'],
                          ['1:50', 'neat', 'fail', '1:50', '1:50'],
                          ['fail', 'neat', 'neat', 'fail', 'fail']])

CRP_matrix = np.array([['1:50', 'neat', 'neat', '1:50', '1:50'],
                       ['1:50', 'neat', 'neat', '1:50', 'fail'],
                       ['1:50', 'neat', 'neat', 'fail', 'fail'],
                       ['1:50', 'neat', 'fail', '1:50', '1:50'],
                       ['fail', 'neat', 'neat', 'fail', 'fail']])

# decisions for various analytes
decisions = {'HRP2_pg_ml': HRP2_matrix, 'LDH_Pan_pg_ml': LDH_Pan_matrix,
             'LDH_Pv_pg_ml': LDH_Pv_matrix, 'CRP_ng_ml': CRP_matrix}

In [11]:
def run_compare(df, analyte_val):#, dil_val):
    above, below, LLQ, ULQ, NA = False, False, False, False, False
    val = df[analyte_val]
    thresh_val = dil_constants[dil_val] * threshholds[analyte_val]
    try:
        float_val = float(val)
        if math.isnan(float_val):
            NA = True
        elif float_val > thresh_val:
            above = True
        elif float_val < thresh_val:
            below = True
    except:
        if '<' in val:
            LLQ = True
        elif '>' in val:
            ULQ = True
    finally:
        return(np.array([above, below, LLQ, ULQ, NA]))

In [12]:
# generate an empty list to fill with small dfs, which will be combined
final_dfs = []
# run counts for decision on what to keep
for analyte in threshholds.keys():
# for analyte in ['HRP2_pg_ml']:
    # create partial function for generating decision vectors
    partial_compare = partial(run_compare, analyte_val = analyte)
    # generate decision vectors
    samples_data['decision_vector'] = samples_data.apply(partial_compare, axis=1)
    # pull decision matrix for given analyte
    decision_matrix = decisions[analyte]
    # generate an empty list to fill with tiny dfs, which will be combined
    tiny_dfs = []
    # iterate over patient_ids
    for i in samples_data['patient_id'].unique().tolist():
        tiny_df = pd.DataFrame(columns=['patient_id', analyte,
                                        '{}_dilution'.format(analyte),
                                        '{}_well'.format(analyte)])
        sub_data = samples_data.loc[samples_data['patient_id'] == i]
        vector_25 = sub_data.loc[sub_data['concentration'].str.contains('2500x'),
                                 'decision_vector'].item()
        vector_125 = sub_data.loc[sub_data['concentration'].str.contains('125000x'),
                                  'decision_vector'].item()
        vector_625 = sub_data.loc[sub_data['concentration'].str.contains('6250000x'),
                                  'decision_vector'].item()
        vector_3125 = sub_data.loc[sub_data['concentration'].str.contains('312500000x'),
                                   'decision_vector'].item()
        decision = decision_matrix[neat_vector, dil_vector].item()
        if decision == '1:50':
            val = sub_data.loc[sub_data['concentration'].str.contains('50x'),
                               analyte].item()
            well = sub_data.loc[sub_data['concentration'].str.contains('50x'),
                               'well'].item()
        elif decision == 'neat':
            val = sub_data.loc[sub_data['concentration'] == '(neat)',
                               analyte].item()
            well = sub_data.loc[sub_data['concentration'] == '(neat)',
                                'well'].item()
        elif decision == 'fail':
            val = np.nan
            well = np.nan
        else:
            raise ValueError("Unexpected decision value: {}".format(val))
        tiny_df = tiny_df.append({'patient_id': i, analyte: val,
                                  '{}_dilution'.format(analyte): decision,
                                  '{}_well'.format(analyte): well}, ignore_index=True)
        tiny_dfs.append(tiny_df)
    small_df = pd.concat(tiny_dfs)
    final_dfs.append(small_df)
output_df = reduce(lambda left, right: pd.merge(left, right, on='patient_id'), final_dfs)
output_df.head(10)

NameError: ("name 'dil_val' is not defined", 'occurred at index 75')